In [34]:
from time import time
from typing import Optional, Sequence
import pandas as pd
import numpy as np

In [35]:
def get_flow_value(cur_solution: list, dist: list, flow: list):
    value = 0
    for i in range(len(cur_solution) - 1):
        for j in range(i + 1, len(cur_solution)):
            value += dist[i][j] * flow[cur_solution[i]][cur_solution[j]]
    value *= 2
    return value

In [36]:
def swap_factories(solution: list, i: int, j: int):
    sol_copy = solution.copy()
    tmp = sol_copy[i]
    sol_copy[i] = sol_copy[j]
    sol_copy[j] = tmp
    return sol_copy

In [37]:
def local_search(n_factories: int, dist: list, flow: list, starting_point: Optional[Sequence[int]] = None):
    '''Local search with first-improvement + don't look bits'''
    start = time()
    if starting_point is None:
        best_sol = [i for i in range(n_factories)]
    else:
        best_sol = starting_point
    best_sol_value = get_flow_value(best_sol, dist, flow)
    while True:
        dlb = [False for i in range(n_factories)]
        improvement_flag = False
        for i in range(n_factories):
            dlb_flag = True
            for j in range(n_factories):
                if not dlb[j] and i != j:
                    new_sol = swap_factories(best_sol, i, j)
                    delta = 0
                    for k in range(n_factories):
                        if k != i and k != j:
                            delta -= dist[i][k] * \
                                flow[best_sol[i]][best_sol[k]]
                            delta -= dist[j][k] * \
                                flow[best_sol[j]][best_sol[k]]
                            delta += dist[i][k] * flow[new_sol[i]][new_sol[k]]
                            delta += dist[j][k] * flow[new_sol[j]][new_sol[k]]
                    delta *= 2
                    if delta > 0:
                        dlb_flag = False
                        improvement_flag = True
                        best_sol_value += delta
                        best_sol = new_sol
            if dlb_flag:
                dlb[i] = True
        if not improvement_flag:
            break

        # print(best_sol, best_sol_value, '\n')
    end = time()

    return best_sol, best_sol_value, end - start

In [38]:
data = []

In [39]:
files_list = ['tai20a.txt', 'tai40a.txt', 'tai60a.txt', 'tai80a.txt', 'tai100a.txt']
 
for file in files_list:       
    with open(f'benchmarks/{file}', 'r') as f:
        dist = []
        flow = []
        n_factories = int(f.readline())
        for i in range(n_factories):
            arr = f.readline().split()
            arr = list(map(int, arr))
            dist.append(arr)
        
        f.readline()
        for i in range(n_factories):
            arr = f.readline().split()
            arr = list(map(int, arr))
            flow.append(arr)
        
        best_sol, best_sol_value, dt = local_search(n_factories, dist, flow)
        data.append([file[:-4] , best_sol_value, dt])
        print(f'Best solution: {best_sol}')
        print(f'Best solution flow value: {best_sol_value}')
        print(f'Time: {dt}\n')
        with open(f'ls_outputs/{file}', 'w') as output_file:
            print(*best_sol, file=output_file)

Best solution: [3, 11, 9, 5, 15, 16, 6, 7, 14, 2, 8, 17, 18, 1, 19, 0, 13, 4, 12, 10]
Best solution flow value: 1043396
Time: 0.013108015060424805

Best solution: [26, 2, 18, 5, 24, 8, 39, 1, 19, 7, 29, 32, 3, 28, 11, 13, 25, 38, 27, 12, 9, 20, 15, 14, 36, 16, 0, 37, 6, 21, 17, 31, 22, 30, 34, 35, 10, 4, 33, 23]
Best solution flow value: 4305536
Time: 0.1753709316253662



Best solution: [40, 8, 21, 14, 27, 17, 1, 35, 34, 0, 44, 59, 30, 2, 13, 16, 58, 6, 57, 3, 54, 4, 7, 22, 37, 10, 12, 25, 26, 23, 20, 18, 36, 33, 32, 11, 46, 19, 38, 5, 41, 31, 42, 43, 29, 39, 56, 51, 9, 48, 50, 28, 52, 53, 47, 55, 45, 24, 49, 15]
Best solution flow value: 9447574
Time: 0.46929144859313965

Best solution: [64, 24, 33, 4, 10, 40, 56, 46, 3, 61, 39, 27, 57, 73, 8, 50, 18, 74, 23, 19, 22, 28, 1, 17, 15, 49, 12, 6, 21, 0, 30, 71, 38, 52, 29, 14, 37, 36, 63, 42, 79, 77, 26, 2, 72, 45, 43, 48, 9, 32, 69, 51, 35, 53, 54, 76, 7, 41, 34, 44, 60, 16, 11, 31, 59, 65, 66, 58, 68, 55, 70, 25, 20, 13, 47, 75, 67, 62, 78, 5]
Best solution flow value: 17250548
Time: 1.1737117767333984

Best solution: [65, 69, 42, 68, 98, 57, 89, 83, 60, 81, 3, 88, 75, 11, 44, 10, 16, 27, 90, 63, 92, 12, 9, 48, 24, 79, 7, 46, 22, 85, 18, 19, 28, 33, 26, 20, 0, 40, 35, 91, 31, 39, 52, 36, 8, 99, 72, 43, 32, 6, 50, 14, 58, 97, 70, 82, 51, 37, 66, 25, 55, 49, 56, 61, 30, 13, 23, 62, 47, 53, 54, 95, 1, 34, 7

In [40]:
ls_df = pd.DataFrame(data, columns=['sample', 'LS best value', 'LS time'])
ls_df

,sample,LS best value,LS time
0,tai20a,1043396,0.013108
1,tai40a,4305536,0.175371
2,tai60a,9447574,0.469291
3,tai80a,17250548,1.173712
4,tai100a,26188816,1.413627


**Iterated Local Search**

In [69]:
data=[]

In [73]:
def inverse_k_perturbator(in_seq: Optional[Sequence[int]] = None, k: Optional[int]=None):
    if in_seq is None:
        return None
    else: 
        l = len(in_seq)
        if k is None:
            p1, p2 = sorted([np.random.randint(0, l), np.random.randint(0, l)])
        else:
            p1 = np.random.randint(0, l)
            p2 = p1 + k % l
            p1,p2 = sorted([p1,p2])
        return in_seq[:p1] + list(reversed(in_seq[p1:p2])) + in_seq[p2:]


In [74]:
def ils(n_factories: int, dist: list, flow: list, 
         perturbation, n_iters = 1, starting_point: Optional[Sequence[int]] = None):
    times = []
    best_sol = starting_point
    best_sol_value = 0
    for i in range(n_iters):
        new_staring_point = perturbation(best_sol)
        new_sol, new_sol_value, dt = local_search(n_factories, dist, flow, new_staring_point)
        times.append(dt)
        if new_sol_value > best_sol_value:
            best_sol_value = new_sol_value
            best_sol = new_sol

    return best_sol, best_sol_value, sum(times)

In [75]:
files_list = ['tai20a.txt', 'tai40a.txt', 'tai60a.txt', 'tai80a.txt', 'tai100a.txt']
 
for file in files_list:       
    with open(f'benchmarks/{file}', 'r') as f:
        dist = []
        flow = []
        n_factories = int(f.readline())
        for i in range(n_factories):
            arr = f.readline().split()
            arr = list(map(int, arr))
            dist.append(arr)
        
        f.readline()
        for i in range(n_factories):
            arr = f.readline().split()
            arr = list(map(int, arr))
            flow.append(arr)
        
        best_sol, best_sol_value, dt = ils(n_factories, dist, flow, inverse_k_perturbator, 10)
        data.append([file[:-4] , best_sol_value, dt])
        print(f'Best solution: {best_sol}')
        print(f'Best solution flow value: {best_sol_value}')
        print(f'Time: {dt}\n')
        with open(f'ils_outputs/{file}', 'w') as output_file:
            print(*best_sol, file=output_file)

Best solution: [3, 7, 9, 18, 0, 13, 6, 15, 14, 2, 11, 8, 16, 10, 19, 1, 5, 4, 12, 17]
Best solution flow value: 1074836
Time: 0.11350893974304199

Best solution: [7, 2, 18, 39, 36, 24, 5, 22, 1, 26, 29, 8, 3, 19, 11, 13, 37, 38, 21, 28, 12, 20, 17, 14, 32, 16, 31, 9, 6, 35, 15, 25, 0, 30, 34, 27, 10, 4, 33, 23]
Best solution flow value: 4326194
Time: 0.9517543315887451

Best solution: [40, 25, 5, 14, 10, 47, 27, 18, 50, 49, 0, 34, 36, 2, 35, 16, 46, 28, 6, 33, 54, 4, 51, 22, 57, 59, 12, 53, 26, 42, 39, 41, 17, 38, 32, 11, 55, 48, 7, 9, 23, 31, 13, 43, 29, 52, 20, 21, 3, 1, 56, 24, 15, 19, 30, 44, 45, 37, 8, 58]
Best solution flow value: 9584336
Time: 4.092381715774536

Best solution: [77, 52, 33, 4, 44, 57, 11, 69, 48, 16, 63, 26, 64, 73, 59, 0, 18, 74, 14, 75, 7, 21, 1, 46, 15, 40, 65, 6, 12, 54, 17, 71, 79, 24, 30, 10, 42, 37, 28, 32, 19, 22, 51, 2, 61, 23, 60, 53, 49, 72, 35, 8, 45, 39, 25, 76, 78, 41, 34, 20, 29, 27, 50, 31, 36, 43, 66, 58, 56, 55, 70, 68, 38, 13, 47, 3, 67, 62, 9,

In [78]:
ils_df = pd.DataFrame(data, columns=['sample', 'ILS best value', 'ILS time'])
df = ls_df.set_index('sample').join(ils_df.set_index('sample'))

In [79]:
df

,LS best value,LS time,ILS best value,ILS time
sample,,,,
tai20a,1043396,0.013108,1074836,0.113509
tai40a,4305536,0.175371,4326194,0.951754
tai60a,9447574,0.469291,9584336,4.092382
tai80a,17250548,1.173712,17344204,7.064329
tai100a,26188816,1.413627,26422338,12.680069
